<a href="https://colab.research.google.com/github/Elcejas77/Ejercicios/blob/main/youtube_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DETECTOR DE GENERO

In [ ]:
#cargar datos desde drive acceso li
FILEID = "1yFZgmduLRmj9j3544uoiU_kQtrdL_r-O"

!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O  canciones.xlsx && rm -rf /tmp/cookies.txt
#!unzip -o codigos.zip
!dir

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_ = 'canciones.xlsx'#leer archivo xlsx con link, band, type
X  = pd.read_excel(file_)
X#imprimir filas iniciales

In [ ]:
#instalar librerias necesarias para descargar audios youtube
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz
!pip install soundfile

!pip install typing-extensions==4.8.0
!pip install --upgrade tensorflow
!pip install fastapi==0.103.0
!pip install pytube
import os
import yt_dlp as youtube_dl

#funcion para descargar mp3 desde youtube
def download_ytvid_as_mp3(video_url,name):
    #video_url = input("enter url of youtube video:")
    video_info = youtube_dl.YoutubeDL().extract_info(url = video_url,download=False)
    filename = f"{name}.mp3"
    options={
        'format':'bestaudio/best',
        'keepvideo':False,
        'outtmpl':filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_info['webpage_url']])

    print("Download complete... {}".format(filename))

In [ ]:
import subprocess
import os

#crear carpeta con resultados
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

#recorrer excel con videos
N, P = X.shape
Ns = N * 5 #cantidad de segmentos por cancion

for n in range(N):
    print(f"video {n+1} de {N}")
    print(f"link: {X.loc[n,'link']}\n")
    print(f"band: {X.loc[n,'band']}\n")
    print(f"type: {X.loc[n,'type']}\n")
    #ruta video n-th
    name_ = 'results/'+X.loc[n,'band']+"_"+str(n)+"_"+str(X.loc[n,'type_num']) # #video+nombre+tipo de genero musical
    #descargar mp3 desde youtube
    download_ytvid_as_mp3(X.loc[n,'link'],name_)
    #convertir a .wav
    subprocess.call(['ffmpeg','-y', '-i', name_+'.mp3',
                   name_+'.wav'])

In [ ]:
#cargar .wavs y partir audios
#lista archivos .wav
path = 'results/'
wav_files = [f for f in os.listdir(path) if f.endswith('.wav')]
wav_files

In [ ]:
!pip install soundfile #instalar sondfile

In [ ]:
import soundfile as sf # para instalar pip install soundfile
#leer archivos y crear np.array audios
fs = 48000
tl = np.array([40,50,60,70,80,90,100]) #puntos lectura
ts = 5 #t segmento
Ns = len(wav_files)*len(tl) #cantidad segmentos
x_t = np.zeros((Ns,int(ts*fs),2)) #Ns segmentos, cantidad de muestras, 2 canales (stereo)
label = np.zeros((Ns,1)) #vector tipo de genero
name_c = []
#leer archivos wav
i = 0
for name in wav_files:#lectura audio .wav
    x, fs = sf.read(path+name)
    for ti in tl: #segmentos de tiempo
        x_t[i] = x[int(fs*ti):int(fs*(ti+ts)),:]
        label[i] = int(name[-5]) #tipo de genero
        name_c += [name[:-6]]
        print(f"{i} lectura: {name}; segundo {ti}:{ti+ts}; tipo música {label[i]}")
        i+=1
x_t.shape

In [ ]:
from IPython.display import Audio #reproducir segmento
i = 10
Audio(x_t[i].T,rate=fs)

In [ ]:
#calculo de fourier
vf = np.fft.rfftfreq(x_t.shape[1],1/fs) #calculo vector de frecuencias
Xw = np.fft.rfft(x_t,axis=1).mean(axis=-1) #transformada rapida de Fourier para señal Real a lo largo del tiempo (axis=1) y se promedian los dos canales
Xw.shape

In [ ]:
#grafica tiempo y fourier
plt.plot(np.arange(0,ts,1/fs),x_t.mean(axis=-1).T) #se promedian los dos canales stereo
plt.xlabel('t[s]')
plt.ylabel('x(t)')
plt.show()

In [ ]:
plt.plot(vf,abs(Xw).T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

In [ ]:
#se normalizan espectros entre 0 y 1 para evitar inconsistencias por ampliltudes máximas
from sklearn.preprocessing import MinMaxScaler
sca = MinMaxScaler()
Xw_ = sca.fit_transform(abs(Xw).T).T

plt.plot(vf,Xw_.T)
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)|')
plt.show()

#en dB
plt.plot(vf,(20*np.log10(Xw_+1e-10)).T) # se suma 1e-10 para evitar discontinuidad del log
plt.xlabel('f[Hz]')
plt.ylabel('|X(f)| dB')
plt.show()

**Nota**: Generalmente el espectro se presenta en [decibeles [dB]](https://es.wikipedia.org/wiki/Decibelio)

In [ ]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
#visualización de datos
red_ = TSNE(perplexity = 20,n_components=2,random_state=123,learning_rate='auto',init='pca')
#red_ = PCA(n_components=2,random_state=123)
fmax = 7000
X_2D = red_.fit_transform(Xw_[:,:fmax]) #se tiene en cuenta el espectro hasta fmax Hz

In [ ]:
#graficar separabilidad 2D
plt.scatter(X_2D[:,0],X_2D[:,1],c=label)
plt.colorbar()
plt.show()

color_ = ["b","y"]
#nombre cancion
plt.scatter(X_2D[:,0],X_2D[:,1],c=label,s=1)
for i, tex in enumerate(name_c):
    #print(f"{i} {tex}")
    plt.text(X_2D[i,0]*1.025,X_2D[i,1]*1.025, tex[:-2]+"_"+str(i), fontsize=6,color=color_[int(label[i]-1)])

#plt.colorbar()
plt.show()

In [ ]:
#reproducir audio
i = 10
Audio(x_t[i].T,rate=fs)

In [ ]:
i = 14
Audio(x_t[i].T,rate=fs)

In [ ]:
import joblib
#guardar modelo
os.mkdir('modelo')
filename_ = 'modelo/reggaeton_vs_metal'
model_ ={'Xw_':Xw_,'fmax': fmax, 'label' : label, 'name_c' : name_c, 'vf':vf,'fs':fs}
joblib.dump(model_,filename_+".pkl")


In [ ]:
#descargar modelo
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'modelo'
shutil.make_archive(namefile, 'zip', 'modelo')
files.download(namefile+'.zip')

#el archivo .zip puede cargarse en drive y utilizarse en otro cuaderno para detectar género musical de nuevos segmentos

In [ ]:
#cargar modelo
my_model_loaded = joblib.load(filename_+".pkl")
my_model_loaded.keys()

EJERCICIO 1 DEL CUADERNO DE COLAB

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

A = 1 # Amplitud
f = 5  # Frecuencia fundamental en Hz
duration = 1 # Duración de la señal en segundos
fs = 1000  # Frecuencia de muestreo en Hz
N = int(fs * duration)  # Número de puntos de muestreo

#Crear una secuencia de tiempo
np.arange(A,f,1/fs)

#Crear la señal sinusoidal
x = A * np.cos(2 * np.pi * f * t)

#Calcular la FFT de la señal
X = np.fft.fft(x)
freq = np.fft.fftfreq(N, 1/fs)

#Graficar la señal en el dominio del tiempo
plt.subplot(2, 1, 1)
plt.plot(t, x)
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Señal en el dominio del tiempo')

#Graficar la magnitud de la FFT (espectro de frecuencia)
plt.subplot(2, 1, 2)
plt.plot(freq, np.abs(X))
plt.xlabel('Frecuencia (Hz)')
plt.ylabel('Magnitud')
plt.title('Espectro de frecuencia (FFT)')

plt.tight_layout()
plt.show()

EJERCICIO 2 DEL CUADERNO DE COLAB

In [ ]:
import numpy as np
import time
import matplotlib.pyplot as plt

# Lista de diferentes longitudes de señal de entrada
lengths = [4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096]

# Listas para almacenar los tiempos de ejecución
time_dft = []
time_fft = []

for length in lengths:
    # Generar una señal de entrada de longitud 'length'
    signal = np.random.rand(length)

    # Calcular la DFT mediante la estimación por sumatoria
    start_time = time.time()
    dft_result = np.fft.fft(signal)
    end_time = time.time()
    time_dft.append(end_time - start_time)

    # Calcular la FFT
    start_time = time.time()
    fft_result = np.fft.fft(signal)
    end_time = time.time()
    time_fft.append(end_time - start_time)

# Crear una gráfica comparativa
plt.plot(lengths, time_dft, label='DFT (Sumatoria)')
plt.plot(lengths, time_fft, label='FFT')
plt.xlabel('Longitud de la señal de entrada')
plt.ylabel('Tiempo de ejecución (s)')
plt.legend()
plt.title('Comparación de Tiempos de Ejecución')
plt.show()

EJERCICIO 3 DEL CUADERNO DE COLAB

In [ ]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

- Se debe incluir el link del video a procesar:

In [ ]:
link="https://www.youtube.com/watch?v=ctb9uuxJVHQ"
!yt-dlp --extract-audio -o "audio" --audio-format mp3 {link}

- Se convierte el audio a .wav para procesar:

In [ ]:
!ffmpeg -y -i audio.mp3 output.wav

Librería para manipulación de archivos de audio

In [ ]:
!pip install soundfile

In [ ]:
import soundfile as sf # para instalar pip install soundfile
#lee archivos wav
nombre_out = "output.wav"
x, fs = sf.read(nombre_out)
# read speech signal from file
print('Frecuencia de muestreo %.2f[Hz]\naudio %s' % (fs,nombre_out))

El audio viene codificado en dos canales:

In [ ]:
x.shape

In [ ]:
from IPython.display import Audio
ns = 30 #se reproducen los primeros ns segundos
Audio(x[:int(fs*ns),:].T,rate=fs)

- Se gráfica un intervalo de la señal en el tiempo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

xpro = x.copy() #copiar archivos para procesar
ti = 10 #tiempo incio a procesar seg
tf = 15 #tiempo final a procesar seg
xs = xpro[int(ti*fs):int((tf*fs)),:]

tt = np.arange(ti,tf,1/fs) # vector de tiempo
plt.plot(tt,xs)
plt.legend(('canal 1','canal 2'))
plt.xlabel('$t[s]$')
plt.ylabel('$x(t)$')
plt.legend()
plt.show()

- Se cálcula el espectro de Fourier del segmento de audio escogido sobre cada canal (sonido estereo).

In [ ]:
Xw = np.fft.rfft(xs,axis=0) # axis=0 permite aplicar fft por cada columna de xpro
#Xwc1 = np.fft.rfft(xs[:,0])
#Xwc2 = np.fft.rfft(xs[:,1])
vf = np.fft.rfftfreq(np.size(xs,0),1/fs) #se crea el vector de frecuencias
plt.plot(vf,abs(Xw))#se grafica la magnitud
plt.legend(('canal 1','canal 2'))
plt.title(r'Espectro audio original')
plt.xlabel(r'$f[Hz]$',fontsize = 14)
plt.ylabel(r'$|X[n]|$',fontsize = 14)
plt.show()

- En este caso, se plantea un filtro pasa banda, apagando los armónicos que no nos interesa.

In [ ]:
#filtrar espectro
Xwf = Xw.copy()
f1 = 500 #frecuencia en Hz corte 1
f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf > f1) & (vf < f2)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwf[ind,:] = 0
plt.plot(vf,abs(Xwf))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro pasa bajos

In [ ]:
#filtrar espectro
Xwfb = Xw.copy()
f1 = 5000 #frecuencia en Hz corte 1
#f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf < f1)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfb[ind,:] = 0
plt.plot(vf,abs(Xwfb))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro pasa altas

In [ ]:
#filtrar espectro
Xwfa = Xw.copy()
f1 = 5000 #frecuencia en Hz corte 1
#f2 = 1000 #frecuencia en Hz corte 2
ind = ~((vf > f1)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfa[ind,:] = 0
plt.plot(vf,abs(Xwfa))
plt.legend(('canal 1','canal 2'))
plt.show()

filtro rechaza banda

In [ ]:
#filtrar espectro
Xwfrr = Xw.copy()
f1 = 100 #frecuencia en Hz corte 1
f2 = 6000#frecuencia en Hz corte 2
ind = ((vf > f1) & (vf < f2)) #frecuencias eliminar-> recueder que ~ actua como negación
Xwfrr[ind,:] = 0
plt.plot(vf,abs(Xwfrr))
plt.legend(('canal 1','canal 2'))
plt.show()

- Ahora, se reconstruye la señal mediante la fft inversa.

In [ ]:
xe2 = np.fft.irfft(Xwf,axis=0) #fft inversa sobre los dos canales de audio
xe2b = np.fft.irfft(Xwfb,axis=0) #fft inversa sobre los dos canales de audio
xe2a = np.fft.irfft(Xwfa,axis=0) #fft inversa sobre los dos canales de audio
xe2r = np.fft.irfft(Xwfrr,axis=0) #fft inversa sobre los dos canales de audio

In [ ]:
Audio(xe2[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa banda

In [ ]:
Audio(xe2b[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa baja

In [ ]:
Audio(xe2a[:int(fs*ns),:].T,rate=fs)#señal del filtro pasa alta

In [ ]:
Audio(xe2r[:int(fs*ns),:].T,rate=fs)#señal del filtro rechaza banda

EJERCICIO 2.5 DEL TALLER

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Parámetros
fs = 1000  # Frecuencia de muestreo (Hz)
T = 1.0    # Duración de la señal (segundos)
t = np.linspace(0, T, int(T * fs), endpoint=False)  # Eje de tiempo

# Señal de mensaje tipo pulso rectangular
def mensaje_pulso_rectangular(t, duty_cycle):
    return np.where(np.mod(t, 1.0) < duty_cycle, 1.0, 0.0)

# Señal de mensaje tipo coseno
def mensaje_coseno(t):
    return np.cos(2 * np.pi * 2 * t)  # Señal coseno de 2 Hz

# Parámetros definidos por el usuario
m = float(input("Ingrese el índice de modulación (entre 0 y 1): "))
duty_cycle = 0.5  # Para la señal de pulso rectangular

# Generación de señales moduladas
mensaje_rectangular = mensaje_pulso_rectangular(t, duty_cycle)
mensaje_cos = mensaje_coseno(t)

portadora = np.cos(2 * np.pi * 10 * t)  # Portadora de 10 Hz

senal_modulada_rectangular = (1 + m * mensaje_rectangular) * portadora
senal_modulada_cos = (1 + m * mensaje_cos) * portadora

# Gráficos en el dominio del tiempo
plt.figure(figsize=(12, 6))
plt.subplot(2, 2, 1)
plt.plot(t, mensaje_rectangular)
plt.title('Señal de mensaje (Pulso Rectangular)')
plt.subplot(2, 2, 2)
plt.plot(t, mensaje_cos)
plt.title('Señal de mensaje (Coseno)')
plt.subplot(2, 2, 3)
plt.plot(t, senal_modulada_rectangular)
plt.title('Señal Modulada (Pulso Rectangular)')
plt.subplot(2, 2, 4)
plt.plot(t, senal_modulada_cos)
plt.title('Señal Modulada (Coseno)')
plt.tight_layout()

# Transformada de Fourier de la señal modulada
def plot_espectro(senal, titulo):
    espectro = np.abs(np.fft.rfft(senal))
    frecuencia = np.fft.rfftfreq(len(senal), 1/fs)

    plt.figure()
    plt.plot(frecuencia, espectro)
    plt.title(titulo)
    plt.xlabel('Frecuencia (Hz)')
    plt.ylabel('Amplitud')

plot_espectro(senal_modulada_rectangular, 'Espectro de la Señal Modulada (Pulso Rectangular)')
plot_espectro(senal_modulada_cos, 'Espectro de la Señal Modulada (Coseno)')

plt.show()